In [1]:
using NearestNeighbors
using WordEmbeddings, SoftmaxClassifier
using Utils
using Query
using Base.Collections
using WordStreams

using Clustering
using Distances
using Iterators

In [3]:
ee = restore("models/ss/text8_200__i5.model");
dtree,labels = nn_tree(ee)

(NearestNeighbors.BallTree{Float32,Query.AngularDist}
  Number of points: 295775
  Dimensions: 200
  Metric: Query.AngularDist()
  Reordered: true,[("rearrangement",1),("piecemeal",1),("quatre",1),("proteases",1),("fsln",1),("duma",1),("libertad",1),("titration",1),("photosynthesis",1),("qajar",1)  …  ("confined",11),("confined",12),("confined",13),("confined",14),("confined",15),("confined",16),("confined",17),("confined",18),("confined",19),("confined",20)])

In [4]:
@time find_nearest_words(ee,"solvent",dtree,labels )

  6.733167 seconds (13.48 M allocations: 488.923 MB, 16.74% gc time)


20-element Array{Array{Tuple{Tuple{String,Int64},Float32},1},1}:
 [(("solvent",1),0.0f0),(("lp",1),0.38617045f0),(("freight",2),0.4005258f0),(("hanukkah",8),0.4013339f0),(("ancestry",1),0.40245205f0)]                    
 [(("solvent",2),0.00010990189f0),(("used",8),0.36280286f0),(("are",13),0.36325294f0),(("sulfuric",1),0.36509895f0),(("chemistry",3),0.36603138f0)]        
 [(("solvent",3),0.0f0),(("panels",4),0.39183682f0),(("material",2),0.3995582f0),(("lifestyles",1),0.4040893f0),(("examination",12),0.40444997f0)]         
 [(("solvent",4),0.0f0),(("adelaide",17),0.39587548f0),(("viking",3),0.40690184f0),(("rpg",16),0.40706158f0),(("achaemenid",1),0.40727332f0)]              
 [(("solvent",5),0.00010990189f0),(("formats",7),0.3888317f0),(("drilling",1),0.3945168f0),(("phase",12),0.39766484f0),(("tank",16),0.399985f0)]           
 [(("solvent",6),0.0f0),(("reactions",11),0.38621718f0),(("quebecers",1),0.3992329f0),(("participated",20),0.399569f0),(("devolved",1),0.39969772f0)]      

In [6]:
using DataStructures



DataStructures.DefaultDict{Int64,Int64,DataType} with 0 entries

DataStructures.DefaultDict{Int64,Int64,F}

In [15]:
x=DefaultDict(AbstractString,DefaultDict{Int64,Int64},()->DefaultDict(Int64,Int64,()->0))

DataStructures.DefaultDict{AbstractString,DataStructures.DefaultDict{Int64,Int64,F},##13#15} with 0 entries

In [19]:
supertype(typeof(x))

Associative{AbstractString,DataStructures.DefaultDict{Int64,Int64,F}}

In [16]:
x["4"][4]

0

In [45]:
function tt()
    1,2,3
end
a,b=tt()
@show a
@show b

a = 1
b = 2


2

In [44]:
keys(aa)

[Set([4,2,3,1])]

In [ ]:
@generated function hash_by_value(x, deep=false)